### Importing Important Libraries For Diease Detection

In [1]:
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

### Importing Datasets From Directory

In [2]:
data_set=tf.keras.preprocessing.image_dataset_from_directory('C:\\Users\\SHYAM\\plant_dataset\\PlantVillage',image_size=(256,256),batch_size=32)

Found 2152 files belonging to 3 classes.


### These data is in form of batches each batch has 32 unit of images with parameter shuffling and has 3 different classes

In [3]:
class_names=data_set.class_names

In [4]:
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

### _________ Define funtion : splitting testing ,validation and training data set________

In [5]:
def test_train_split(data,train_size=0.8,test_size=0.1,val_score=.10,shuffle=True,shuffle_size=10000):
    
    
    if shuffle:
        data=data.shuffle(shuffle_size,seed=12)
        
    train_size1=int(len(data)*train_size)
    test_size1=int(len(data)*test_size)
    valid_size=int(len(data)*val_score)
        
    ds_train=data.take(train_size1)
    ds_test=data.skip(train_size1).take(test_size1)
    ds_validate=data.skip(train_size1).skip(valid_size)
    
    return ds_train,ds_test,ds_validate

In [6]:
x_train,x_test,x_validate=test_train_split(data_set)

#### With use of Funtion  __Prefetch() and __cache() we improve time and efficiency of Model and                                                   __buffer_size tells how many batch of file will get loaded beforehand in to CPU in this case its autotuned according to tf pipeline

In [7]:
x_train=x_train.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
x_test=x_test.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [9]:
x_validate=x_validate.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)

## __Image preprocessing such  as ___Resizing() , ___Rescaling(), ___flipping(), and__Rotating() 

In [10]:
data_resize_rescale=tf.keras.Sequential([
   keras.layers.experimental.preprocessing.Resizing(256,256),
   keras.layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [11]:
data_augumatation=tf.keras.Sequential([
   keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
   keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

### Making CNN MODEL

In [12]:
model=keras.models.Sequential([data_resize_rescale,
                              data_augumatation,
                              keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(32,256,256,3)),
                              keras.layers.MaxPooling2D((2,2)),
                              keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu') ,
                              keras.layers.MaxPooling2D((2,2)),
                              keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
                              keras.layers.MaxPooling2D((2,2)),
                              keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
                              keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
                              keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
                              keras.layers.MaxPooling2D((2,2)),
                               keras.layers.Flatten(),
                               keras.layers.Dense(64,activation='relu'),
                              keras.layers.Dense(3,activation='softmax') ,
                              ])

In [13]:
model.build((32,256,256,3))

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                  

In [15]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [16]:
history=model.fit(x_train,epochs=10,batch_size=32,verbose=1,validation_data=x_validate)

Epoch 1/10
54/54 [==============================] - 163s 3s/step - loss: 0.9232 - accuracy: 0.5110 - val_loss: 0.8591 - val_accuracy: 0.5078
Epoch 2/10
54/54 [==============================] - 126s 2s/step - loss: 0.7276 - accuracy: 0.6817 - val_loss: 0.5884 - val_accuracy: 0.7539
Epoch 3/10
54/54 [==============================] - 142s 3s/step - loss: 0.4970 - accuracy: 0.7951 - val_loss: 0.5093 - val_accuracy: 0.8008
Epoch 4/10
54/54 [==============================] - 134s 2s/step - loss: 0.3592 - accuracy: 0.8692 - val_loss: 0.4540 - val_accuracy: 0.8320
Epoch 5/10
54/54 [==============================] - 119s 2s/step - loss: 0.2616 - accuracy: 0.9051 - val_loss: 0.2481 - val_accuracy: 0.8867
Epoch 6/10
54/54 [==============================] - 134s 2s/step - loss: 0.2424 - accuracy: 0.9045 - val_loss: 0.1684 - val_accuracy: 0.9297
Epoch 7/10
54/54 [==============================] - 126s 2s/step - loss: 0.1804 - accuracy: 0.9271 - val_loss: 0.1695 - val_accuracy: 0.9180
Epoch 8/10
54

In [18]:
predicted=model.predict(x_test)

6/6 [==============================] - 2s 304ms/step


In [44]:
type(predicted)

numpy.ndarray

### checking for Prediction of test data

In [49]:
for i in range(len(predicted)):
    print(class_names[np.argmax(predicted[i])])

Potato___Late_blight
Potato___Early_blight
Potato___Late_blight
Potato___Early_blight
Potato___Early_blight
Potato___Early_blight
Potato___Late_blight
Potato___Early_blight
Potato___Early_blight
Potato___Late_blight
Potato___Late_blight
Potato___Early_blight
Potato___Late_blight
Potato___Early_blight
Potato___Early_blight
Potato___Late_blight
Potato___Late_blight
Potato___Late_blight
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Potato___Late_blight
Potato___Early_blight
Potato___Early_blight
Potato___Early_blight
Potato___Early_blight
Potato___Early_blight
Potato___Late_blight
Potato___Early_blight
Potato___Early_blight
Potato___Late_blight
Potato___Late_blight
Potato___Early_blight
Potato___Early_blight
Potato___Late_blight
Potato___Late_blight
Potato___Early_blight
Potato___Late_blight
Potato___Early_blight
Potato___Early_blight
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Potato___Late_blight
Potato___Early_blight
Potato___Late_blight
Potato___Early_bli